In [1]:
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

dir0 = Path('el_nino/')
file_sst = 'sst.mnmean.nc'
file_2 = 'mslp_coarse.nc'

# Load the data set with xarray
ds_nino = xr.open_dataset(Path(dir0, file_sst))
ds_mslp = xr.open_dataset(Path(dir0, file_2))

# Define 3.4 region
lat_min, lat_max = -5.5, 5.5
lon_min, lon_max = 190, 240

# Interpolating to get rid of the nan-values
ds_nino = ds_nino.interpolate_na(dim='lon')
ds_mslp = ds_mslp.interpolate_na(dim='lon')

# Select the region
ds_region_nino = ds_nino.where((ds_nino.lat >= lat_min) & (ds_nino.lat <= lat_max) & 
                               (ds_nino.lon >= lon_min) & (ds_nino.lon <= lon_max), drop=True)
ds_region_mslp = ds_mslp.where((ds_mslp.latitude >= lat_min) & (ds_mslp.latitude <= lat_max) & 
                               (ds_mslp.longitude >= lon_min) & (ds_mslp.longitude <= lon_max), drop=True)

In [2]:
# Extracting the labels from 01/1982 to 05/2021
# -1 = La Nina
# 0 = Nothing
# 1 = El Nino

# Initialisation
start_date_y = pd.Timestamp(year = 1982, month = 1, day = 1)
end_date_y = pd.Timestamp(year = 2021, month = 5, day = 1)
current_date = start_date_y

# Mean temperature in the region over all the years
big_mean = float(ds_region_nino.mean()['sst'])

ys = []

while current_date <= end_date_y:

    # Create timestamps for previous, current, and next months
    current_month = current_date
    prev_month = current_month - pd.DateOffset(months = 1)
    next_month = current_month + pd.DateOffset(months = 1)

    # Get data for each month
    ds_prev_month = ds_region_nino.sel(time = slice(prev_month, prev_month))
    ds_curr_month = ds_region_nino.sel(time = slice(current_month, current_month))
    ds_next_month = ds_region_nino.sel(time = slice(next_month, next_month))

    # Merge the three datasets
    merged_dataset = xr.concat([ds_prev_month, ds_curr_month, ds_next_month], dim='time')

    # Calculate the average sea surface temperature anomaly
    sst_anom = float(merged_dataset['sst'].mean()) - big_mean
    # print(current_date, ': ', sst_anom)

    cases = [
        (sst_anom >= 0.5),
        (sst_anom < 0.5) & (sst_anom > -0.5),
        (sst_anom <= -0.5)
    ]
    conditions = [1, 0, -1]
    res = np.select(cases, conditions, 0)

    ys.append(res)
    
    # Increment to the first day of the next month
    current_date += pd.DateOffset(months = 1)

# Convert the list to a numpy array
ys_np = np.array(ys)

In [36]:
# How many month in advance do we want to make predictions
n_month = 8

# Dataset to predict n_month in advance using 1 year of data
start_date_X = start_date_y - pd.DateOffset(years = 1) - pd.DateOffset(months = n_month - 1)
end_date_X = end_date_y - pd.DateOffset(years = 1) - pd.DateOffset(months = n_month - 1)
current_date = start_date_X

xs_np = {}

while current_date <= end_date_X:
    
    start_variable = current_date
    end_variable = current_date + pd.DateOffset(years = 1) - pd.DateOffset(months = 1)
    # print(start_variable, ' => ', end_variable)

    # Selecting the data for the one-year interval
    interval_data = ds_mslp.sel(time = slice(start_variable, end_variable))

    # Formatting the interval data
    numpy_array = interval_data['msl'].to_numpy()
    flattened_data = numpy_array.flatten()
    xs_np[str(end_variable.year) + "/" + str(end_variable.month + n_month)] = flattened_data

    # Increment to the first day of the next month
    current_date += pd.DateOffset(months = 1)

xs_np = np.array(list(xs_np.values()))

In [6]:
# First model :

X_train, X_test, y_train, y_test = train_test_split(xs_np, ys_np, test_size = 0.2, random_state = 42)

# Create a pipeline with preprocessing and SVM
model_1 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components = 0.80)),
    ('lda', LinearDiscriminantAnalysis()),
    ('svm', SVC(kernel='poly'))
])

# Train the model
model_1.fit(X_train, y_train)

# Make predictions
y_pred_test = model_1.predict(X_test)
y_pred_train = model_1.predict(X_train)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})

# Display the DataFrame
print(comparison_df)

# Evaluate the model
accuracy_test = accuracy_score(y_test, y_pred_test)
accuracy_train = accuracy_score(y_train, y_pred_train)

print(f"Train Accuracy: {accuracy_train}")
print(f"Test Accuracy: {accuracy_test}")

    Actual  Predicted
0        0          0
1        0          0
2       -1         -1
3        0          0
4        1          1
..     ...        ...
90      -1         -1
91       0          0
92       1          1
93       0          0
94      -1         -1

[95 rows x 2 columns]
Train Accuracy: 0.8544973544973545
Test Accuracy: 0.7684210526315789


In [37]:
# Second Model :

model_2 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components = 0.90)), 
    ('lda', LinearDiscriminantAnalysis(solver = 'eigen', shrinkage = 0.5)),
    ('svm', SVC(kernel = 'rbf', C = 0.1, gamma = 0.2))
])

# Stratified K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

# Cross-validation scores
scores = cross_val_score(model_2, xs_np, ys_np, cv = kfold)

# Fit the model on the entire dataset
model_2.fit(xs_np, ys_np)

print("Cross-validation scores:", scores)
print("Average score:", np.mean(scores))

Cross-validation scores: [0.71578947 0.50526316 0.56842105 0.61702128 0.64893617]
Average score: 0.6110862262038074


In [21]:
# Prediction using the model

# Which time intervall do we want to predict
pred_start_date_y = pd.Timestamp(year = 2022, month = 1, day = 1)
pred_end_date_y = pd.Timestamp(year = 2022, month = 12, day = 1)

# Dataset to make the prediction
pred_start_date_X = pred_start_date_y - pd.DateOffset(years = 1) - pd.DateOffset(months = n_month - 1)
pred_end_date_X = pred_end_date_y - pd.DateOffset(years = 1) - pd.DateOffset(months = n_month - 1)
current_date_pred = pred_start_date_X

xs_np_pred = {}

# Extracting the data we will use for the prediction
while current_date_pred <= pred_end_date_X:

    start_variable = current_date_pred
    end_variable = current_date_pred + pd.DateOffset(years = 1) - pd.DateOffset(months = 1)
    # print(start_variable, ' => ', end_variable)

    # Selecting the data for the one-year interval
    interval_data = ds_mslp.sel(time=slice(start_variable, end_variable))

    # Formatting the interval data
    numpy_array = interval_data['msl'].to_numpy()
    flattened_data = numpy_array.flatten()
    xs_np_pred[str(end_variable.year) + "/" + str(end_variable.month + n_month)] = flattened_data

    # Going to the next month
    current_date_pred += pd.DateOffset(months = 1)

xs_np_pred = np.array(list(xs_np_pred.values()))

pred_2022 = model_2.predict(xs_np_pred)

print(pred_2022)

[-1 -1 -1  0  0  0  0 -1 -1 -1 -1 -1]


In [14]:
# Neural Network

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(xs_np)

# Apply PCA
pca = PCA(n_components =  0.95)
X_pca = pca.fit_transform(X_scaled)

# Splitting the code in Test and Training set
X_train, X_test, y_train, y_test = train_test_split(X_pca, ys_np, test_size = 0.2,shuffle = True, random_state = 42)

# Apply LDA
lda = LinearDiscriminantAnalysis(solver = 'eigen', shrinkage = 0.5)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Encode the labels
y_train = to_categorical(y_train, num_classes = 3)
y_test = to_categorical(y_test, num_classes = 3)

# To be used for the first Layer of the neural network
rows, cols = X_train.shape
print(cols)

# L1 Regularization factor
l1_lambda = 0.01

# Create an optimizer
optimizer = SGD(learning_rate = 0.2, momentum = 0.4)

# Create a Sequential model for regression
model_nn = Sequential()

# Shape of the neural network
model_nn.add(Dense(4, activation = 'relu', input_shape = (cols,), kernel_regularizer = regularizers.l1(l1_lambda)))
model_nn.add(Dense(4, activation = 'relu', kernel_regularizer = regularizers.l1(l1_lambda)))
model_nn.add(Dense(4, activation = 'relu', kernel_regularizer = regularizers.l1(l1_lambda)))
model_nn.add(Dense(3, activation = 'softmax', kernel_regularizer = regularizers.l1(l1_lambda)))

model_nn.compile(optimizer = optimizer, 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

history = model_nn.fit(X_train, y_train, epochs = 15, validation_data=(X_test, y_test))

# Evaluate the model_nn
test_loss, test_accuracy = model_nn.evaluate(X_test, y_test)
train_loss, train_accuracy = model_nn.evaluate(X_train, y_train)

print(f"Train loss: {train_loss}")
print(f"Test loss: {test_loss}")
print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

2
Epoch 1/15
12/12 [==============================] - 1s 21ms/step - loss: 1.1072 - accuracy: 0.6243 - val_loss: 1.0211 - val_accuracy: 0.6421
Epoch 2/15
12/12 [==============================] - 0s 4ms/step - loss: 0.7118 - accuracy: 0.8677 - val_loss: 0.9126 - val_accuracy: 0.6842
Epoch 3/15
12/12 [==============================] - 0s 3ms/step - loss: 0.5471 - accuracy: 0.9180 - val_loss: 0.8888 - val_accuracy: 0.7368
Epoch 4/15
12/12 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.9286 - val_loss: 0.8738 - val_accuracy: 0.7263
Epoch 5/15
12/12 [==============================] - 0s 3ms/step - loss: 0.4387 - accuracy: 0.9392 - val_loss: 0.9097 - val_accuracy: 0.7263
Epoch 6/15
12/12 [==============================] - 0s 3ms/step - loss: 0.4189 - accuracy: 0.9339 - val_loss: 0.9125 - val_accuracy: 0.7368
Epoch 7/15
12/12 [==============================] - 0s 5ms/step - loss: 0.3968 - accuracy: 0.9339 - val_loss: 0.8745 - val_accuracy: 0.7789
Epoch 8/15
12/12 